# Reddit API

In [27]:
from soc_api import Reddit
import pandas as pd

pd.set_option('display.max_columns', None)

In [ ]:
client_id = ''
client_secret = ''

red = Reddit(client_id,client_secret)

In [ ]:
post_ids = [p for p in red.get_submissions('MLS',time_filter='week', limit=1000)]
print(len(post_ids))
post_ids[0:3]

In [ ]:
thread_urls = ['https://www.reddit.com/r/MLS/comments/138w5p7/lizzy_becherano_exclusive_sources_confirm_a_new/',
              'https://www.reddit.com/r/MLS/comments/138ts1p/tannenwald_the_full_transcript_of_the_exchange/']

coms = red.process_submissions(thread_urls)

In [ ]:
dates = [c['created_date'] for c in coms]


In [31]:
# filename = 'MLS_{}.csv'.format('_'.join((min(dates).strftime(format='%Y-%m-%d'), max(dates).strftime(format='%Y-%m-%d'))))
# red.write_csv(coms,filename)
coms_df = pd.read_csv('MLS_2023-04-30_2023-05-07.csv')

In [ ]:
# coms_df = pd.DataFrame(coms)
coms_df['score_percent'] = coms_df.score/coms_df.score.sum()
print(coms_df.shape)
coms_df.head()

In [ ]:
# 
pd.Series([a[0] for a in coms_df.parent_id.str.split('_').dropna()]).value_counts()b

In [ ]:
coms_df.type.value_counts()

In [ ]:
coms_df.url_domain.value_counts()

In [ ]:
coms_df[coms_df.score>2].score.hist()

In [ ]:
coms_df[coms_df.score>2].score.hist(by=coms_df['type'])

In [ ]:
coms_df \
    .groupby([pd.to_datetime(coms_df.created_date).dt.date,'type']).count()['id'].reset_index() \
    .pivot(index='created_date', columns='type', values='id') \
    .plot.bar(subplots=True)

In [ ]:
coms_df \
    .groupby([pd.to_datetime(coms_df.created_date).dt.date,'type']).sum()['score'].reset_index() \
    .pivot(index='created_date', columns='type', values='score') \
    .plot.bar(subplots=True)

In [ ]:
coms_df.gilded.sum()

In [ ]:
com

In [ ]:
coms_df[coms_df.controversiality !=0].head()

In [ ]:
coms_df.groupby('retrieve_time').count().head()['id']


In [ ]:
reddit = red.session# auth
posts = reddit.subreddit('MLS').top(time_filter='day', limit=1000)

In [ ]:
ps = [p for p in posts]

In [ ]:
ps[0].id

In [ ]:
len(ps)

# LLM Model API - Huggingface

In [79]:
def ask_hf_llm(q,m,token):
    '''
    Query a hugging face hosted llm model.
    '''
    import requests

    API_URL = "https://api-inference.huggingface.co/models/{}".format(m)
    headers = {"Authorization": "Bearer {}".format(token)}

    def query(payload):
        response = requests.post(API_URL, headers=headers, json=payload)
        return response.json()

    output = query({
        "inputs": q,
    })
    return output

q = 'What are you?'
for m in ["databricks/dolly-v2-3b", "gpt-3.5-turbo", "google/flan-t5-large","bigscience/bloomz"]:
    print(m,': ', ask_hf_llm(q,m,'xxx'))
    

databricks/dolly-v2-3b :  [{'generated_text': 'What are you?\n\nI am a data scientist and machine learning enthusiast. I have been'}]
gpt-3.5-turbo :  {'error': 'Model gpt-3.5-turbo does not exist'}
google/flan-t5-large :  [{'generated_text': 'a sailor'}]
bigscience/bloomz :  [{'generated_text': 'What are you? a film'}]


# LLM Model API - Langchain

For huggingface hosted models only supports [`text2text generation`](https://huggingface.co/models?pipeline_tag=text2text-generation&sort=downloads) and [`textgeneration`](https://huggingface.co/models?pipeline_tag=text-generation&sort=downloads) types.

In [21]:

import os 
 
os.environ["HUGGINGFACEHUB_API_TOKEN"] = ''

os.environ["OPENAI_API_KEY"] = ''

In [81]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain
from langchain.llms import HuggingFacePipeline
from langchain.chat_models import ChatOpenAI
import torch
from transformers import pipeline

# load a few models
model_params = {"temperature":0, "max_length":64}
# generate_text = pipeline(model="databricks/dolly-v2-3b", torch_dtype=torch.bfloat16,
#                          trust_remote_code=True, device_map="auto", return_full_text=True)
# llm_dolly_local = HuggingFacePipeline(pipeline=generate_text) # locally saved dolly pipeline
llm_dolly = HuggingFaceHub(repo_id="databricks/dolly-v2-3b", model_kwargs=model_params) # hf hosted llm
llm = ChatOpenAI(model_name="gpt-3.5-turbo") # openai hosted llm
llm_flan = HuggingFaceHub(repo_id="google/flan-t5-large", model_kwargs=model_params) # hf hosted llm
llm_bloom = HuggingFaceHub(repo_id="bigscience/bloomz") # hf hosted llm

# define the prompt
template = """ Question: {question}"""
prompt = PromptTemplate(template=template, input_variables=["question"])

# create chains
llm_dolly_chain = LLMChain(prompt=prompt,llm=llm_dolly)
llm_chain = LLMChain(prompt=prompt, llm=llm)
llm_chain_flan = LLMChain(prompt=prompt, llm=llm_flan)
llm_chain_bloom = LLMChain(prompt=prompt, llm=llm_bloom)

# run
question = "Who is the best soccer player in history?"

print(question,'\n')
print('# Dolly:\n',llm_dolly_chain.run(question).strip())
print('\n# gpt-3.5-turbo:\n',llm_chain.run(question))
print('\n# Flan:\n',llm_chain_flan.run(question))
print('\n# Bloom:\n',llm_chain_bloom.run(question))

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)).


Who is the best soccer player in history? 

# Dolly:
 The answer is complicated. There are many ways to measure the best soccer player in history.

One way is to compare the number of goals scored. The all-time top scorer is Lionel Messi. He has scored 634 goals in

# gpt-3.5-turbo:
 As an AI language model, I cannot have personal opinions, but according to many experts and fans around the world, the best soccer player in history is widely regarded as Pelé, Lionel Messi, or Diego Maradona. However, this is a subjective topic and opinions may vary.

# Flan:
 gertrude bell

# Bloom:
  Pele


# Pandas DataFrame Agent

In [64]:
from langchain.agents import create_pandas_dataframe_agent

df = pd.read_csv('simulated.csv')
agent = create_pandas_dataframe_agent(llm, coms_df, verbose=True)

agent.run("how many rows and columns are there?")



> Entering new AgentExecutor chain...
Thought: We can use the `shape` attribute of the dataframe to find the number of rows and columns.
Action: python_repl_ast
Action Input: df.shape
Observation: (9715, 24)
Thought:The dataframe has 9715 rows and 24 columns.
Final Answer: There are 9715 rows and 24 columns.

> Finished chain.


'There are 9715 rows and 24 columns.'

In [ ]:
agent.run("""Use python_repl_ast tool. Create a TFIDF matrix using the body column and cluster into topics. Convert body column to lowercase.""")

In [119]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
df = coms_df

# Remove any rows with NaN values in the body column
df.dropna(subset=['body'], inplace=True)

# Convert text to lowercase
df['body'] = df['body'].str.lower()

# Remove punctuation
df['body'] = df['body'].str.replace('[^\w\s]','')

# Remove numbers
df['body'] = df['body'].str.replace('\d+', '')

# Remove stop words
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
df['body'] = df['body'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

# Create TFIDF matrix
tfidf_vectorizer = TfidfVectorizer()
tfidf = tfidf_vectorizer.fit_transform(df['body'])

# Cluster into topics
kmeans = KMeans(n_clusters=5).fit(tfidf)

C:\Users\twnba\Anaconda3\envs\llm\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [123]:
pd.Series(kmeans.labels_).value_counts()

1    6332
4    1745
2     628
3     450
0     380
Name: count, dtype: int64

# Custom Dataframe Agent

# Topic Modelling (Clustering)


In [110]:
coms_df['content'] = coms_df.title.fillna('') + coms_df.submission_text.fillna('') + coms_df.body.fillna('')

## Embeddings

In [120]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.chains.summarize import load_summarize_chain

hf_embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

In [111]:
coms_df['source'] = coms_df.id

loader = DataFrameLoader(coms_df, page_content_column="content")
docs = loader.load()
text_split = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 0)
text = text_split.split_documents(docs)

In [112]:
text[0]

Document(page_content='[meme] It would end the pain', metadata={'subreddit': 'MLS', 'submission_text': nan, 'created_utc': 1682945845.0, 'controversiality': nan, 'submission_id': '134ls7m', 'score': 1109, 'likes': nan, 'distinguished': nan, 'downs': nan, 'title': '[meme] It would end the pain', 'author': 'Fraganade', 'upvote_ratio': 0.98, 'url_domain': 'i.imgur.com', 'permalink': '/r/MLS/comments/134ls7m/meme_it_would_end_the_pain/', 'retrieve_time': '2023-05-06 23:17:30', 'created_date': '2023-05-01 12:57:25', 'parent_id': nan, 'type': 'submission', 'gilded': nan, 'url': 'https://i.imgur.com/duojFS6.jpg', 'body': nan, 'num_reports': nan, 'total_awards_received': nan, 'id': '134ls7m', 'source': '134ls7m'})

In [113]:
len(text)

9845

In [114]:
coms_df.shape

(9715, 26)

In [115]:
# pass the text and embeddings to FAISS
vectorstore = FAISS.from_documents(text, hf_embeddings)

In [ ]:
# vectorstore.save_local('faiis_db')
vectorstore.write_index('faiss_db.index')

In [ ]:
from pprint import pprint
query = "Philadelphia?"
documents = vectorstore.similarity_search(query)
pprint(documents)

In [ ]:
len(documents)

In [ ]:
question = 'What are the main news?'
my_chain = load_qa_with_sources_chain(llm, chain_type="refine")
my_chain({"input_documents": documents, "question": question})

In [ ]:
prompt_template = """Write a concise summary of the following:


{text}


CONCISE SUMMARY:"""
PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])
chain = load_summarize_chain(llm_dolly, chain_type="stuff", prompt=PROMPT)
chain.run(text)

In [133]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectorstore.as_retriever())
qa.run('Can you give me insisghts?')

"I'm sorry, I cannot provide insights as your question is too broad and lacks context. Please provide more specific information or ask a more specific question."

## Clustering